parsePubmedAbstracts

Mary Kate Montgomery

11/17/21


Script to parse search results exported from PubMed in "PubMed" text format. Pulls PMID values and Abstracts (if the entry has one) then saves them to a csv.

In [90]:
# Import libraries
import pandas as pd
import numpy as np

In [91]:
# Read text file
topic = 'Pacemaker'
with open(topic+'/PubMed-'+topic+'.txt') as f:
    lines = f.readlines()

In [92]:
ct = -1
keys = []; vals = []
for l in lines:
    if '- ' in l:
        key = l.split('- ')[0]
        keys.append(key)
        vals.append(l.replace(key+'- ','').replace('\n',''))
        ct += 1
    else:
        vals[ct] = vals[ct] + l.replace('\n','').strip() + ' '
pb = pd.DataFrame({'key':keys,'val':vals})

In [93]:
# Get just PMIDs and Abstracts
pmids = []; abss = []; id_locs = []
# Get locs of PMIDs
for l in range(len(pb)):
    if pb.loc[l,'key'] == 'PMID':
        id_locs.append(l)
# Get PMIDs and ABs
for i in range(len(id_locs)-1):
    pmids.append(pb.loc[id_locs[i],'val'])
    found = False
    for j in range(id_locs[i],id_locs[i+1]):
        if pb.loc[j,'key'] == 'AB  ':
            abss.append(pb.loc[j,'val'])
            found = True
    # Handle items w/ no abstract
    if found == False:
        abss.append('')

In [94]:
# Put in dataframe
df_abs = pd.DataFrame({'PMID':pmids,'Abstract':abss})
df_abs

,PMID,Abstract
0,31070324,The Micra® leadless pacemaker has demonstrated...
1,31373521,Aims: Novel leadless pacemakers (LPMs) may red...
2,30893006,
3,30936325,Runaway pacemaker is phenomenon in which pacem...
4,28956076,BACKGROUND: Most outpatient follow-ups after p...
...,...,...
9348,28304219,BACKGROUND: Although transcatheter aortic-valv...
9349,30883053,BACKGROUND: Transcatheter aortic-valve replace...
9350,32913013,OBJECTIVE: To define the risks and consequence...
9351,28613276,Reduced cardiac vagal control reflected in low...


In [ ]:
# Read rest of data
df_pub = pd.read_csv(topic+'/PubMed-'+topic+'.csv')
df_pub = df_pub.set_index('PMID')
df_pub.head()

In [ ]:
# Join together
df_full = df_pub.join(df_abs)
df_full.head()

In [ ]:
# Write to file
df_full.to_csv(topic+'/PubMed_wAbstracts-'+topic+'.csv')